In [21]:
import olympusair
import os
import datetime

In [22]:
cam = olympusair.Camera()
cam.commInterface()

cam.switchMode('standalone')
cam.setProperty('RAW','ON')
cam.setProperty('TAKE_DRIVE','DRIVE_NORMAL')
cam.setProperty('TAKEMODE','P')
cam.setProperty('DESTINATION_FILE','DESTINATION_FILE_MEDIA')

# cam.switchMode('play')
# cam.switchMode('playmaintenance')
# print cam.getFilesList()
# cam.removeFileProtection()
# cam.removeAllFiles()
# print cam.getFilesList()
# cam.switchMode('standalone')
# cam.switchMode('rec')

Camera init: OK
Establishing camera event notification: OK
Establishing event socket, port 65000:  OK
Camera communication mode change to wifi: OK
Camera mode change to standalone: OK
Camera property RAW read ON (OFF ON): OK
Camera property RAW change to ON: Not needed
Camera property TAKE_DRIVE read DRIVE_NORMAL (DRIVE_NORMAL DRIVE_CONTINUE): OK
Camera property TAKE_DRIVE change to DRIVE_NORMAL: Not needed
Camera property TAKEMODE read P (iAuto P A S M ART movie Genius): OK
Camera property TAKEMODE change to P: Not needed
Camera property DESTINATION_FILE read DESTINATION_FILE_MEDIA (DESTINATION_FILE_MEDIA): OK
Camera property DESTINATION_FILE change to DESTINATION_FILE_MEDIA: Not needed


In [23]:
cam.switchMode('rec')

destPath = os.path.join('/','media','pi','REEF')
if not os.path.exists(destPath):
    os.makedirs(destPath)

startTime = datetime.datetime.now()
while(1):
        elapsed = datetime.datetime.now()
        if elapsed - startTime > datetime.timedelta(seconds=15):
            
            print '>> %s: Acquiring a photo' % elapsed.strftime('%Y-%m-%d_%H-%M-%S')
            
            cam.startPreview()
            cam.takePicture()
            cam.waitOnCardWrite()
            cam.stopPreview()
            
            jpeg, raw = cam.getLatestFile()
            
            
            dirPath =  os.path.join(destPath,elapsed.strftime('%Y-%m-%d_%H-%M-%S'))
            if not os.path.exists(dirPath):
                os.mkdir(dirPath)
            
            jpegFile = open(os.path.join(dirPath,'IMG.jpg'),'w')
            jpegFile.write(jpeg)
            
            rawFile = open(os.path.join(dirPath,'IMG.orf'),'w')
            rawFile.write(raw)
            
            close(jpegFile)
            close(rawFile)
            
            startTime = elapsed
        

Camera mode change to rec: OK
>> 2016-07-26_10-25-18: Acquiring a photo
Establishing liveview socket 65001: OK
Camera live view resolution change to 0320x0240: OK
Camera live view start: OK
Camera picture acquisition: OK
Camera live view stop: OK


TypeError: expected a character buffer object

In [ ]:
cam.disconnect()

In [24]:
jpegList, rawList = cam.getFilesList()

In [25]:
print jpegList

[(/DCIM/100OLYMP, P1010001.JPG, 7901543, 0, 2015-01-01_00-00-00), (/DCIM/100OLYMP, P1010002.JPG, 7901531, 0, 2015-01-01_00-00-00), (/DCIM/100OLYMP, P1010003.JPG, 7935527, 0, 2015-01-01_00-00-00), (/DCIM/100OLYMP, P1010004.JPG, 7952821, 0, 2015-01-01_00-00-00), (/DCIM/100OLYMP, P1010005.JPG, 7924516, 0, 2015-01-01_00-00-00), (/DCIM/100OLYMP, P1010006.JPG, 7921988, 0, 2015-01-01_00-00-00), (/DCIM/100OLYMP, P1010007.JPG, 7936191, 0, 2015-01-01_00-00-00), (/DCIM/100OLYMP, P1010008.JPG, 7939435, 0, 2015-01-01_00-00-00), (/DCIM/100OLYMP, P1010009.JPG, 7647782, 0, 2015-01-01_00-00-00), (/DCIM/100OLYMP, P1010010.JPG, 7783291, 0, 2015-01-01_00-00-00), (/DCIM/100OLYMP, P1010011.JPG, 7777048, 0, 2015-01-01_00-00-00), (/DCIM/100OLYMP, P1010012.JPG, 7796193, 0, 2015-01-01_00-00-00), (/DCIM/100OLYMP, P1010013.JPG, 7732942, 0, 2015-01-01_00-00-00), (/DCIM/100OLYMP, P1010014.JPG, 7798510, 0, 2015-01-01_00-00-00)]


In [26]:
print rawList

[]


In [ ]:
rawList.sort(reverse = True)